In [ ]:
from lightglue import LightGlue, SuperPoint, viz2d, DISK, SIFT, ALIKED, DoGHardNet
from lightglue.utils import load_image, rbd, match_pair
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import torch
import torchvision.transforms.functional as TF
import os
import cv2
import pandas as pd  # Import pandas
import glob
import math
from tqdm.notebook import tqdm

general_folder_path = '/home/oussama/Documents/EPFL/PDS_LUTS/'

output_path = general_folder_path + 'panorama.jpg'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.cuda.empty_cache()

# Utility function to calculate image corners in homogeneous coordinates
def get_homogeneous_corners(width, height):
    return np.array([[0, 0, 1], [width, 0, 1], [width, height, 1], [0, height, 1]]).T

def warp_perspective_padded1(src, dst, transf):
    src_h, src_w = src.shape[:2]
    dst_h, dst_w = dst.shape[:2]

    # Define the corners of the src image
    src_corners = np.array([
        [0, 0],
        [src_w, 0],
        [src_w, src_h],
        [0, src_h]
    ], dtype=np.float32)

    # Transform the src corners using the homography matrix (transf)
    src_corners_transformed = cv2.perspectiveTransform(src_corners[None, :, :], transf)[0]

    # Define the corners of the dst image in its own coordinate space
    dst_corners = np.array([
        [0, 0],
        [dst_w, 0],
        [dst_w, dst_h],
        [0, dst_h]
    ], dtype=np.float32)

    # Combine all corners to find the overall bounding box
    all_corners = np.vstack((src_corners_transformed, dst_corners))

    # Compute the bounding box of all corners
    x_min, y_min = np.int32(all_corners.min(axis=0))
    x_max, y_max = np.int32(all_corners.max(axis=0))

    # Calculate the translation needed to shift images to positive coordinates
    shift_x = -x_min
    shift_y = -y_min

    # Compute the size of the output canvas
    output_width = x_max - x_min
    output_height = y_max - y_min

    # Compute the 3x3 translation matrix to shift the images
    translation_matrix = np.array([
        [1, 0, shift_x],
        [0, 1, shift_y],
        [0, 0,      1]
    ], dtype=np.float32)

    # Update the transformation matrix to include the translation
    new_transf = translation_matrix @ transf

    # Warp the src image using the updated transformation matrix
    warped = cv2.warpPerspective(src, new_transf, (output_width, output_height))
    
    # Warp the dst image using only the translation matrix (affine)
    dst_pad = cv2.warpAffine(dst, translation_matrix[:2], (output_width, output_height))

    # Determine the anchor points
    anchorX = int(shift_x)
    anchorY = int(shift_y)

    # Determine the sign
    sign = (anchorX > 0) or (anchorY > 0)

    return dst_pad, warped, anchorX, anchorY, sign

# Rotate image tensor by specified angle
def rotate_image(image, angle):
    return TF.rotate(image, angle)

def rotate_image1(image, angle):
    """
    Rotate a PyTorch image tensor without cropping and add necessary padding to preserve all content.
    Keeps the tensor on its original device (CPU or CUDA).
    Args:
        image: A PyTorch tensor in CHW format with values in [0, 1].
        angle: Rotation angle in degrees (counterclockwise).
    Returns:
        Rotated image as a PyTorch tensor on the same device.
    """
    device = image.device  # Store original device (CPU or CUDA)

    # Convert tensor to NumPy array (HWC format)
    if isinstance(image, torch.Tensor):
        image_np = image.permute(1, 2, 0).cpu().numpy()  # CHW -> HWC on CPU
        if image_np.max() <= 1:  # Scale up to [0, 255] if needed
            image_np = (image_np * 255).astype(np.uint8)
    else:
        raise TypeError("Input must be a PyTorch tensor in CHW format.")

    # Get the height and width of the image
    h, w = image_np.shape[:2]
    center = (w // 2, h // 2)

    # Compute the rotation matrix and new dimensions
    rotation_matrix = cv.getRotationMatrix2D(center, angle, 1.0)
    cos_val = abs(rotation_matrix[0, 0])
    sin_val = abs(rotation_matrix[0, 1])
    new_w = int((h * sin_val) + (w * cos_val))
    new_h = int((h * cos_val) + (w * sin_val))

    # Adjust the rotation matrix to account for translation
    rotation_matrix[0, 2] += (new_w / 2) - center[0]
    rotation_matrix[1, 2] += (new_h / 2) - center[1]

    # Perform rotation with padding
    rotated_image_np = cv.warpAffine(
        image_np, rotation_matrix, (new_w, new_h),
        flags=cv.INTER_CUBIC,
        borderMode=cv.BORDER_CONSTANT,
        borderValue=(0, 0, 0)
    )

    # Convert back to PyTorch tensor (CHW format)
    rotated_image_tensor = torch.from_numpy(rotated_image_np).permute(2, 0, 1).float()
    if rotated_image_tensor.max() > 1:  # Normalize back to [0, 1]
        rotated_image_tensor /= 255.0

    # Move the tensor back to the original device
    return rotated_image_tensor.to(device)

# Calculate similarity score between two images
def compute_similarity_score(image1, image2, extractor, matcher):
    with torch.no_grad():
        feats0 = extractor.extract(image1)
        feats1 = extractor.extract(image2)
        feats0, feats1, matches01 = match_pair(extractor, matcher, image1, image2)
    points0 = feats0['keypoints'][matches01['matches'][..., 0]]
    score = points0.shape[0]
    # Release GPU memory
    del feats0, feats1, matches01, points0
    torch.cuda.empty_cache()
    return score

def split_image_paths(image_paths):
    """
    Splits the image_paths list into two halves:
    - First half gets an extra frame if the total number of images is odd.
    - Second half is reversed to start from the last image and go to the middle.

    Args:
        image_paths (list): List of image paths.

    Returns:
        tuple: (first_half, second_half) - two lists of image paths.
    """
    # Compute the midpoint
    mid = (len(image_paths) + 1) // 2

    # Split the list
    first_half = image_paths[:mid]  # First half
    second_half = image_paths[mid:][::-1]  # Second half, reversed

    return first_half, second_half


In [ ]:
# Feature extractor and matcher initialization
extractor = DoGHardNet(max_num_keypoints=None).eval().cuda()
matcher = LightGlue(features='doghardnet').eval().cuda()

start = True
general_folder_path = '/home/oussama/Documents/EPFL/PDS_LUTS/'
image_folder = os.path.join(general_folder_path, 'images_updated')

indice = 0

# Get all .jpg files in the folder
image_paths = glob.glob(os.path.join(image_folder, '*.jpg'))
# Optionally, sort the image paths if order matters
image_paths.sort()

image_paths = image_paths[indice:] + image_paths[:indice]


In [ ]:
# Load images
image_path0  = image_paths[0]

# Save first image
cv.imwrite(general_folder_path+'warped_image.jpg', cv.imread(image_path0))

image0 = load_image(image_path0)

# Initialize DataFrame for image corners
image_corners_df = pd.DataFrame(columns=['image_path', 'corners', 'frame_number'])

# Add first image's corners to the DataFrame
first_image_corners = np.array([[0, 0], [image0.shape[2]-1, 0],
                                [image0.shape[2]-1, image0.shape[1]-1],
                                [0, image0.shape[1]-1]], dtype=np.int32)

frame_number = os.path.splitext(os.path.basename(image_path0))[0]

new_row = pd.DataFrame({
    'image_path': [image_path0],
    'corners': [first_image_corners],
    'frame_number': [frame_number]
})

image_corners_df = pd.concat([image_corners_df, new_row], ignore_index=True)
first_half, second_half = split_image_paths(image_paths)


In [ ]:
num = 0
best = 0

for idx in tqdm(range(indice + 1, len(first_half))):
    # image_path0 = general_folder_path + 'anchor_test.jpg'
    if not start:
        image_path0  = general_folder_path + 'warped_image.jpg'

    image0 = load_image(image_path0)

    image_path1  = first_half[idx]

    image1 = load_image(image_path1)

    # Determine best rotation angle
    rotation_angles = range(0, 360, 45)
    best_score, best_angle = -1, 0

    for angle in rotation_angles:
        rotated_image1 = rotate_image(image1, angle)
        score = compute_similarity_score(image0, rotated_image1, extractor, matcher)
        if score > best_score:
            best_score, best_angle = score, angle
    best_rotated_image = rotate_image(image1, best_angle)
    # Prepare images for final stitching
    imocv0 = cv.imread(image_path0)
    imocv1 = cv.cvtColor(np.array(TF.to_pil_image(best_rotated_image.cpu()).convert("RGB")), cv.COLOR_RGB2BGR)
    cv.imwrite(general_folder_path + 'rotated_image.jpg', imocv1)

    # Extract keypoints and compute homography matrix
    feats0, feats1, matches01 = match_pair(extractor, matcher, image0, best_rotated_image)
    points0 = feats0['keypoints'][matches01['matches'][..., 0]].cpu().numpy()
    points1 = feats1['keypoints'][matches01['matches'][..., 1]].cpu().numpy()

    if points0.shape[0] >= 4:
        M, _ = cv.findHomography(points1, points0, cv.RANSAC, 5.0)
        
        if num % 1 == 0:
            M_normalized = M / M[2, 2]

            # Extract the rotation and translation components
            M = np.array([
                [M_normalized[0, 0], M_normalized[0, 1], M_normalized[0, 2]],
                [M_normalized[1, 0], M_normalized[1, 1], M_normalized[1, 2]],
                [M_normalized[2, 0],                 0,                 1]
            ])
        num += 1
        # angle_rad = math.atan2(M_normalized[1, 0], M_normalized[0, 0])  # Returns the angle in radians
        angle_rad = math.atan2(M[1, 0], M[0, 0])  # Returns the angle in radians
        angle_deg = math.degrees(angle_rad) - best_angle
        dst_padded, warped_image, anchorX1, anchorY1, sign = warp_perspective_padded1(imocv1, imocv0, M)

        before_last_key = image_corners_df['image_path'].iloc[-1]
        warped_image_corners = image_corners_df.loc[image_corners_df['image_path'] == before_last_key, 'corners'].values[0]
        x_coords, y_coords = warped_image_corners[:, 0], warped_image_corners[:, 1]

        # Define source corners as a numpy array of four points
        corners = np.array([[x_coords[0], y_coords[0]],
                            [x_coords[1], y_coords[1]],
                            [x_coords[2], y_coords[2]],
                            [x_coords[3], y_coords[3]]], dtype=np.float32)

        b_x_min, b_y_min = np.min(corners, axis=0).astype(int)
        b_x_max, b_y_max = np.max(corners, axis=0).astype(int)

        # Define source corners as a numpy array and convert to float32
        new_image_corners = np.array([[0, 0], [imocv1.shape[1], 0],
                                    [imocv1.shape[1], imocv1.shape[0]],
                                    [0, imocv1.shape[0]]], dtype=np.float32)

        # Perform perspective transform with correctly typed data
        transformed_corners = cv.perspectiveTransform(np.array([new_image_corners], dtype=np.float32), M)[0]

        # Update image_corners_df
        # adjusted_corners = transformed_corners + [b_x_min, b_y_min]
        adjusted_corners = transformed_corners + [anchorX1, anchorY1]
        new_row = pd.DataFrame({
            'image_path': [image_path1],
            'corners': [adjusted_corners],
            'frame_number': [os.path.splitext(os.path.basename(image_path1))[0]]
        })
        image_corners_df = pd.concat([image_corners_df, new_row], ignore_index=True)

        if start:
            idx0 = image_corners_df[image_corners_df['image_path'] == image_path0].index[0]
            image_corners_df.at[idx0, 'corners'] += [anchorX1, anchorY1]
            (anchorX, anchorY) = (0, 0)

        # Overlay warped image onto padded destination
        non_zero_mask = (warped_image > 0).astype(np.uint8)
        dst_padded[non_zero_mask == 1] = warped_image[non_zero_mask == 1]

        # Get last and before last keys
        last_key = image_corners_df['image_path'].iloc[-1]
        warped_image_corners = image_corners_df.loc[image_corners_df['image_path'] == last_key, 'corners'].values[0]
        x_coords, y_coords = warped_image_corners[:, 0], warped_image_corners[:, 1]

        # Define source corners as a numpy array of four points
        corners = np.array([[x_coords[0], y_coords[0]],
                            [x_coords[1], y_coords[1]],
                            [x_coords[2], y_coords[2]],
                            [x_coords[3], y_coords[3]]], dtype=np.float32)

        x_min, y_min = np.min(corners, axis=0).astype(int)
        x_max, y_max = np.max(corners, axis=0).astype(int)

        image_corners_0 = image_corners_df.loc[image_corners_df['image_path'] == before_last_key, 'corners'].values[0]
        image_corners_1 = image_corners_df.loc[image_corners_df['image_path'] == last_key, 'corners'].values[0]

        # Crop the region of interest
        warped_image = warped_image[y_min:y_max, x_min:x_max]
        cv.imwrite(general_folder_path + 'warped_image.jpg', warped_image)

        # # Crop the region of interest
        # image2 = load_image(first_half[idx])
        # image2 = rotate_image(image2, -angle_deg)
        # warped_image = image2
        # cv.imwrite(general_folder_path+'warped_image.jpg', cv.cvtColor(np.array(TF.to_pil_image(image2.cpu()).convert("RGB")), cv.COLOR_RGB2BGR))

        # Save final images
        cv.imwrite(general_folder_path + 'panorama.jpg', dst_padded)
    else:
        print("Not enough points to compute homography.")

    if start:
        cv.imwrite(general_folder_path+'aligned_image.jpg', cv.imread(general_folder_path+'panorama.jpg'))

    # Load the images
    current_panorama = cv.imread(general_folder_path + 'panorama.jpg')
    new_image = cv.imread(general_folder_path + 'aligned_image.jpg')

    # Create a 3x3 translation matrix
    translation_matrix = np.float32([
        [1, 0, b_x_min - anchorX1],
        [0, 1, b_y_min - anchorY1],
        [0, 0, 1]
    ])

    if start:
        # Create a 3x3 translation matrix Identity matrix
        translation_matrix = np.float32([
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1]
        ])

    # Apply the translation to the images
    dst_padded, warped_image, anchorX, anchorY, _ = warp_perspective_padded1(current_panorama, new_image, translation_matrix)
    idx = image_corners_df[image_corners_df['image_path'] == last_key].index[0]
    if not start:
        image_corners_df.at[idx, 'corners'] += [b_x_min - anchorX1 + anchorX, b_y_min -anchorY1 + anchorY]
    for img_path in image_corners_df['image_path']:
        if img_path != image_path1:
            idx = image_corners_df[image_corners_df['image_path'] == img_path].index[0]
            image_corners_df.at[idx, 'corners'] += [anchorX, anchorY]
    start = False
    # # Update image_corners for image_path1
    # idx1 = image_corners_df[image_corners_df['image_path'] == image_path1].index[0]
    # image_corners_df.at[idx1, 'corners'] = transformed_corners + [anchorX, anchorY]
    # image_corners_df.at[idx1, 'corners'] = image_corners_df.at[idx1, 'corners'].astype(np.int32)

    # Create a mask where dst_padded has zero pixels
    mask_dst = cv.cvtColor(dst_padded, cv.COLOR_BGR2GRAY)
    mask_dst = (mask_dst == 0).astype(np.uint8)

    # Ensure mask has three channels
    mask_dst_3ch = cv.merge([mask_dst, mask_dst, mask_dst])

    # Combine images by filling zeros in dst_padded with pixels from warped_image
    combined_image = dst_padded.copy()
    combined_image[mask_dst_3ch == 1] = warped_image[mask_dst_3ch == 1]

    # Save the combined image
    cv.imwrite(general_folder_path + 'aligned_image.jpg', combined_image)


In [ ]:
indice = 0

start = True

num = 0

for idx in tqdm(range(indice + 1, len(second_half))):
    # image_path0 = general_folder_path + 'anchor_test.jpg'
    if not start:
        image_path0  = general_folder_path + 'warped_image.jpg'
    else:
        image_path0  = image_paths[0]

    image0 = load_image(image_path0)

    image_path1  = second_half[idx]

    image1 = load_image(image_path1)

    # Determine best rotation angle
    rotation_angles = range(0, 360, 45)
    best_score, best_angle = -1, 0

    for angle in rotation_angles:
        rotated_image1 = rotate_image(image1, angle)
        score = compute_similarity_score(image0, rotated_image1, extractor, matcher)
        if score > best_score:
            best_score, best_angle = score, angle
            best_rotated_image = rotated_image1

    # Prepare images for final stitching
    imocv0 = cv.imread(image_path0)
    imocv1 = cv.cvtColor(np.array(TF.to_pil_image(best_rotated_image.cpu()).convert("RGB")), cv.COLOR_RGB2BGR)

    # Extract keypoints and compute homography matrix
    feats0, feats1, matches01 = match_pair(extractor, matcher, image0, best_rotated_image)
    points0 = feats0['keypoints'][matches01['matches'][..., 0]].cpu().numpy()
    points1 = feats1['keypoints'][matches01['matches'][..., 1]].cpu().numpy()

    if points0.shape[0] >= 4:
        M, _ = cv.findHomography(points1, points0, cv.RANSAC, 5.0)
        
        if num % 1 == 0:
            M_normalized = M / M[2, 2]

            # Extract the rotation and translation components
            M = np.array([
                [M_normalized[0, 0], M_normalized[0, 1], M_normalized[0, 2]],
                [M_normalized[1, 0], M_normalized[1, 1], M_normalized[1, 2]],
                [M_normalized[2, 0],                 0,                 1]
            ])
        num += 1
        # angle_rad = math.atan2(M_normalized[1, 0], M_normalized[0, 0])  # Returns the angle in radians
        angle_rad = math.atan2(M[1, 0], M[0, 0])  # Returns the angle in radians
        angle_deg = math.degrees(angle_rad) + best_angle

        dst_padded, warped_image, anchorX1, anchorY1, sign = warp_perspective_padded1(imocv1, imocv0, M)

        if start:
            before_last_key = image_corners_df['image_path'].iloc[0]
            warped_image_corners = image_corners_df.loc[image_corners_df['image_path'] == before_last_key, 'corners'].values[0]
            x_coords, y_coords = warped_image_corners[:, 0], warped_image_corners[:, 1]
        else : 
            before_last_key = image_corners_df['image_path'].iloc[-1]
            warped_image_corners = image_corners_df.loc[image_corners_df['image_path'] == before_last_key, 'corners'].values[0]
            x_coords, y_coords = warped_image_corners[:, 0], warped_image_corners[:, 1]

        # Define source corners as a numpy array of four points
        corners = np.array([[x_coords[0], y_coords[0]],
                            [x_coords[1], y_coords[1]],
                            [x_coords[2], y_coords[2]],
                            [x_coords[3], y_coords[3]]], dtype=np.float32)

        b_x_min, b_y_min = np.min(corners, axis=0).astype(int)
        b_x_max, b_y_max = np.max(corners, axis=0).astype(int)

        # Define source corners as a numpy array and convert to float32
        new_image_corners = np.array([[0, 0], [imocv1.shape[1], 0],
                                    [imocv1.shape[1], imocv1.shape[0]],
                                    [0, imocv1.shape[0]]], dtype=np.float32)

        # Perform perspective transform with correctly typed data
        transformed_corners = cv.perspectiveTransform(np.array([new_image_corners], dtype=np.float32), M)[0]

        # Update image_corners_df
        # adjusted_corners = transformed_corners + [b_x_min, b_y_min]
        adjusted_corners = transformed_corners + [anchorX1, anchorY1]
        new_row = pd.DataFrame({
            'image_path': [image_path1],
            'corners': [adjusted_corners],
            'frame_number': [os.path.splitext(os.path.basename(image_path1))[0]]
        })
        image_corners_df = pd.concat([image_corners_df, new_row], ignore_index=True)
        
        # Overlay warped image onto padded destination
        non_zero_mask = (warped_image > 0).astype(np.uint8)
        dst_padded[non_zero_mask == 1] = warped_image[non_zero_mask == 1]

        # Get last and before last keys
        last_key = image_corners_df['image_path'].iloc[-1]
        warped_image_corners = image_corners_df.loc[image_corners_df['image_path'] == last_key, 'corners'].values[0]
        x_coords, y_coords = warped_image_corners[:, 0], warped_image_corners[:, 1]

        # Define source corners as a numpy array of four points
        corners = np.array([[x_coords[0], y_coords[0]],
                            [x_coords[1], y_coords[1]],
                            [x_coords[2], y_coords[2]],
                            [x_coords[3], y_coords[3]]], dtype=np.float32)

        x_min, y_min = np.min(corners, axis=0).astype(int)
        x_max, y_max = np.max(corners, axis=0).astype(int)

        image_corners_0 = image_corners_df.loc[image_corners_df['image_path'] == before_last_key, 'corners'].values[0]
        image_corners_1 = image_corners_df.loc[image_corners_df['image_path'] == last_key, 'corners'].values[0]

        # Crop the region of interest
        warped_image = warped_image[y_min:y_max, x_min:x_max]
        cv.imwrite(general_folder_path + 'warped_image.jpg', warped_image)

        # image2 = load_image(second_half[idx])
        # image2 = rotate_image1(image2, -angle_deg)
        # warped_image = image2
        # cv.imwrite(general_folder_path+'warped_image.jpg', cv.cvtColor(np.array(TF.to_pil_image(image2.cpu()).convert("RGB")), cv.COLOR_RGB2BGR))

        # Save final images
        cv.imwrite(general_folder_path + 'panorama.jpg', dst_padded)
    else:
        print("Not enough points to compute homography.")

    # Load the images
    current_panorama = cv.imread(general_folder_path + 'panorama.jpg')
    new_image = cv.imread(general_folder_path + 'aligned_image.jpg')

    # Create a 3x3 translation matrix
    translation_matrix = np.float32([
        [1, 0, b_x_min - anchorX1],
        [0, 1, b_y_min - anchorY1],
        [0, 0, 1]
    ])

    # Apply the translation to the images
    dst_padded, warped_image, anchorX, anchorY, _ = warp_perspective_padded1(current_panorama, new_image, translation_matrix)
    idx = image_corners_df[image_corners_df['image_path'] == last_key].index[0]
    image_corners_df.at[idx, 'corners'] += [b_x_min - anchorX1 + anchorX, b_y_min -anchorY1 + anchorY]
    for img_path in image_corners_df['image_path']:
        if img_path != image_path1:
            idx = image_corners_df[image_corners_df['image_path'] == img_path].index[0]
            image_corners_df.at[idx, 'corners'] += [anchorX, anchorY]

    start = False

    # Create a mask where dst_padded has zero pixels
    mask_dst = cv.cvtColor(dst_padded, cv.COLOR_BGR2GRAY)
    mask_dst = (mask_dst == 0).astype(np.uint8)

    # Ensure mask has three channels
    mask_dst_3ch = cv.merge([mask_dst, mask_dst, mask_dst])

    # Combine images by filling zeros in dst_padded with pixels from warped_image
    combined_image = dst_padded.copy()
    combined_image[mask_dst_3ch == 1] = warped_image[mask_dst_3ch == 1]

    # Save the combined image
    cv.imwrite(general_folder_path + 'aligned_image.jpg', combined_image)

cv.imwrite(general_folder_path + 'final_panorama.jpg', combined_image)


In [ ]:
plt.figure(figsize=(40, 30))
# Update and display the panorama with corner points
plt.imshow(cv.cvtColor(combined_image, cv.COLOR_BGR2RGB))
for _, row in image_corners_df.iterrows():
    corners = row['corners']
    x_coords, y_coords = corners[:, 0], corners[:, 1]
    plt.plot(x_coords, y_coords, 'o-', label=os.path.basename(row['image_path']))
    plt.fill(x_coords, y_coords, alpha=0.3)
plt.axis('on')
# plt.legend()  # Uncomment if you want to add a legend
plt.show()

# Save the df to a csv file
image_corners_df.to_csv(general_folder_path + 'results/image_corners.csv', index=False)

torch.cuda.empty_cache()



In [ ]:
#############################################
# Imports and Configuration
#############################################

import os
import glob
import cv2
import cv2 as cv
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
from tqdm.notebook import tqdm
import math
from PIL import Image

from lightglue import LightGlue, DoGHardNet
from lightglue.utils import load_image, match_pair

# Set paths and constants
GENERAL_FOLDER_PATH = '/home/oussama/Documents/EPFL/PDS_LUTS/'
IMAGE_FOLDER = os.path.join(GENERAL_FOLDER_PATH, 'images_updated')
PANORAMA_OUTPUT_PATH = os.path.join(GENERAL_FOLDER_PATH, 'panorama.jpg')
RESULTS_FOLDER = os.path.join(GENERAL_FOLDER_PATH, 'results')
if not os.path.exists(RESULTS_FOLDER):
    os.makedirs(RESULTS_FOLDER)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

# Utility functions
def rotate_image(image, angle):
    """Rotate a PIL image tensor by a given angle."""
    return TF.rotate(image, angle)

def rotate_image1(image, angle):
    """
    Rotate an image without cropping and add necessary padding to preserve all content.

    Args:
        image: A PIL Image or PyTorch tensor.
        angle: Rotation angle in degrees (counterclockwise).

    Returns:
        Rotated image with padding to fit all content.
    """
    # Convert to PIL Image if it's a tensor
    if isinstance(image, torch.Tensor):
        image = TF.to_pil_image(image)

    # Get original dimensions
    w, h = image.size

    # Calculate new canvas size to fit the rotated image
    angle_rad = math.radians(angle)
    new_w = int(abs(w * math.cos(angle_rad)) + abs(h * math.sin(angle_rad)))
    new_h = int(abs(w * math.sin(angle_rad)) + abs(h * math.cos(angle_rad)))

    # Create a larger blank canvas with padding
    canvas = Image.new("RGBA", (new_w, new_h), (0, 0, 0, 0))

    # Calculate the translation to center the original image
    translation_x = (new_w - w) // 2
    translation_y = (new_h - h) // 2

    # Paste the original image into the center of the blank canvas
    canvas.paste(image, (translation_x, translation_y))

    # Rotate the entire canvas
    rotated = canvas.rotate(angle, resample=Image.BICUBIC, expand=False)

    # Convert back to RGB if the original image was RGB
    if image.mode == "RGB":
        rotated = rotated.convert("RGB")

    return rotated

def compute_similarity_score(image1, image2, extractor, matcher):
    """Compute similarity score between two images based on matched keypoints."""
    with torch.no_grad():
        feats0, feats1, matches01 = match_pair(extractor, matcher, image1, image2)
    points0 = feats0['keypoints'][matches01['matches'][..., 0]]
    score = points0.shape[0]
    del feats0, feats1, matches01, points0
    torch.cuda.empty_cache()
    return score

def split_image_paths(image_paths):
    """
    Splits the image_paths list into two halves:
    - First half gets an extra frame if the total number of images is odd.
    - Second half is reversed to start from the last image and go to the middle.

    Args:
        image_paths (list): List of image paths.

    Returns:
        tuple: (first_half, second_half) - two lists of image paths.
    """
    # Compute the midpoint
    mid = (len(image_paths) + 1) // 2

    # Split the list
    first_half = image_paths[:mid]  # First half
    second_half = image_paths[mid:][::-1]  # Second half, reversed

    return first_half, second_half

def warp_perspective_padded(src, dst, transf):
    """Warp 'src' image into 'dst' image using a homography 'transf' with padding."""
    src_h, src_w = src.shape[:2]
    dst_h, dst_w = dst.shape[:2]

    src_corners = np.float32([[0, 0],
                              [src_w, 0],
                              [src_w, src_h],
                              [0, src_h]])
    dst_corners = np.float32([[0, 0],
                              [dst_w, 0],
                              [dst_w, dst_h],
                              [0, dst_h]])

    src_corners_transformed = cv2.perspectiveTransform(src_corners[None, :, :], transf)[0]
    all_corners = np.vstack((src_corners_transformed, dst_corners))

    x_min, y_min = np.int32(all_corners.min(axis=0))
    x_max, y_max = np.int32(all_corners.max(axis=0))

    shift_x = -x_min
    shift_y = -y_min
    output_width = x_max - x_min
    output_height = y_max - y_min

    translation_matrix = np.array([
        [1, 0, shift_x],
        [0, 1, shift_y],
        [0, 0, 1]
    ], dtype=np.float32)

    new_transf = translation_matrix @ transf
    warped = cv2.warpPerspective(src, new_transf, (output_width, output_height))
    dst_pad = cv2.warpAffine(dst, translation_matrix[:2], (output_width, output_height))

    anchorX = int(shift_x)
    anchorY = int(shift_y)
    sign = (anchorX > 0) or (anchorY > 0)
    return dst_pad, warped, anchorX, anchorY, sign

#############################################
# Initialization
#############################################

# Feature extractor and matcher
extractor = DoGHardNet(max_num_keypoints=None).eval().cuda()
matcher = LightGlue(features='doghardnet').eval().cuda()

start = True



In [ ]:
# import math
# import numpy as np
# from PIL import Image
# import torchvision.transforms.functional as TF
# import cv2 as cv
# import matplotlib.pyplot as plt

# def rotate_image_with_padding(image, angle):
#     """
#     Rotate an image without cropping and add necessary padding to preserve all content.

#     Args:
#         image: A PIL Image or PyTorch tensor.
#         angle: Rotation angle in degrees (counterclockwise).

#     Returns:
#         Rotated image with padding to fit all content.
#     """
#     # Convert to PIL Image if it's a tensor
#     if isinstance(image, torch.Tensor):
#         image = TF.to_pil_image(image)

#     # Get original dimensions
#     w, h = image.size

#     # Calculate new canvas size to fit the rotated image
#     angle_rad = math.radians(angle)
#     new_w = int(abs(w * math.cos(angle_rad)) + abs(h * math.sin(angle_rad)))
#     new_h = int(abs(w * math.sin(angle_rad)) + abs(h * math.cos(angle_rad)))

#     # Create a larger blank canvas with padding
#     canvas = Image.new("RGBA", (new_w, new_h), (0, 0, 0, 0))

#     # Calculate the translation to center the original image
#     translation_x = (new_w - w) // 2
#     translation_y = (new_h - h) // 2

#     # Paste the original image into the center of the blank canvas
#     canvas.paste(image, (translation_x, translation_y))

#     # Rotate the entire canvas
#     rotated = canvas.rotate(angle, resample=Image.BICUBIC, expand=False)

#     # Convert back to RGB if the original image was RGB
#     if image.mode == "RGB":
#         rotated = rotated.convert("RGB")

#     return rotated
# image10 = load_image(image_paths[-1])
# image10 = rotate_image_with_padding(image10, 10)
# image10 = cv.cvtColor(np.array(image10), cv.COLOR_RGB2BGR)
# plt.imshow(cv.cvtColor(image10, cv.COLOR_BGR2RGB))


In [ ]:
#############################################
# Get all images to stitch
#############################################

# Load all images
image_paths = glob.glob(os.path.join(IMAGE_FOLDER, '*.jpg'))
image_paths.sort()  # Ensure consistent order

# Select the first image as a reference
indice = 0
# image_paths = image_paths[indice:] + image_paths[:indice]
image_path0 = image_paths[indice]

# Save and load the first image in both cv and tensor form
cv.imwrite(os.path.join(GENERAL_FOLDER_PATH, 'warped_image.jpg'), cv.imread(image_path0))
image0 = load_image(image_path0)
imocv0 = cv.imread(image_path0)

# Prepare DataFrame to store image corners and frame numbers
image_corners_df = pd.DataFrame(columns=['image_path', 'corners', 'frame_number'])

# Compute corners of the first image
first_image_corners = np.array([
    [0, 0],
    [imocv0.shape[1]-1, 0],
    [imocv0.shape[1]-1, imocv0.shape[0]-1],
    [0, imocv0.shape[0]-1]
], dtype=np.int32)

frame_number = os.path.splitext(os.path.basename(image_path0))[0]

# Add the first image's corners directly
new_row = pd.DataFrame({
    'image_path': [image_path0],
    'corners': [first_image_corners],
    'frame_number': [frame_number]
})
image_corners_df = pd.concat([image_corners_df, new_row], ignore_index=True)# Define the maximum number of images to process (for demonstration)
# For a real scenario, you can use: for idx in range(indice+1, len(image_paths)):
for idx in tqdm(range(0 + 1, len(image_paths))):
    if not start:
        # After the first iteration, image0 is the "warped_image.jpg"
        image_path0 = os.path.join(GENERAL_FOLDER_PATH, 'warped_image.jpg')

    image0 = load_image(image_path0)
    image_path1 = image_paths[idx]
    image1 = load_image(image_path1)

    # Determine best rotation angle
    rotation_angles = range(0, 360, 45)
    best_score, best_angle = -1, 0
    best_rotated_image = None

    for angle in rotation_angles:
        rotated_image1 = rotate_image(image1, angle)
        score = compute_similarity_score(image0, rotated_image1, extractor, matcher)
        if score > best_score:
            best_score = score
            best_angle = angle
            best_rotated_image = rotated_image1

    imocv0 = cv.imread(image_path0)
    imocv1 = cv.cvtColor(np.array(TF.to_pil_image(best_rotated_image.cpu()).convert("RGB")), cv.COLOR_RGB2BGR)

    feats0, feats1, matches01 = match_pair(extractor, matcher, image0, best_rotated_image)
    points0 = feats0['keypoints'][matches01['matches'][..., 0]].cpu().numpy()
    points1 = feats1['keypoints'][matches01['matches'][..., 1]].cpu().numpy()

    if points0.shape[0] < 4:
        print("Not enough points to compute homography for image:", image_path1)
        continue

    M, _ = cv.findHomography(points1, points0, cv.RANSAC, 5.0)
    M_normalized = M / M[2, 2]
    M = np.array([
        [M_normalized[0, 0], M_normalized[0, 1], M_normalized[0, 2]],
        [M_normalized[1, 0], M_normalized[1, 1], M_normalized[1, 2]],
        [0,                  0,                  1]
    ])

    angle_rad = math.atan2(M_normalized[1, 0], M_normalized[0, 0])  # Returns the angle in radians
    angle_deg = math.degrees(angle_rad) + best_angle
    print(f"Rotation angle: {angle_deg:.2f} degrees")

    dst_padded, warped_image, anchorX1, anchorY1, sign = warp_perspective_padded(imocv1, imocv0, M)

    # If not the very first iteration, retrieve corners from the previous image
    if not start:
        before_last_key = image_corners_df['image_path'].iloc[-1]
        warped_image_corners = image_corners_df.loc[
            image_corners_df['image_path'] == before_last_key, 'corners'
        ].values[0]
    else:
        # If it's the start, just use the first image's corners
        warped_image_corners = image_corners_df.loc[
            image_corners_df['image_path'] == image_path0, 'corners'
        ].values[0]

    x_coords, y_coords = warped_image_corners[:, 0], warped_image_corners[:, 1]
    prev_corners = np.array([
        [x_coords[0], y_coords[0]],
        [x_coords[1], y_coords[1]],
        [x_coords[2], y_coords[2]],
        [x_coords[3], y_coords[3]]
    ], dtype=np.float32)

    b_x_min, b_y_min = np.min(prev_corners, axis=0).astype(int)
    b_x_max, b_y_max = np.max(prev_corners, axis=0).astype(int)

    new_image_corners = np.array([
        [0, 0],
        [imocv1.shape[1], 0],
        [imocv1.shape[1], imocv1.shape[0]],
        [0, imocv1.shape[0]]
    ], dtype=np.float32)

    transformed_corners = cv2.perspectiveTransform(np.array([new_image_corners], dtype=np.float32), M)[0]
    adjusted_corners = transformed_corners + [anchorX1, anchorY1]

    # Add new image corners
    new_row = pd.DataFrame({
        'image_path': [image_path1],
        'corners': [adjusted_corners],
        'frame_number': [os.path.splitext(os.path.basename(image_path1))[0]]
    })
    image_corners_df = pd.concat([image_corners_df, new_row], ignore_index=True)

    if start:
        # Adjust the first image's corners once we have anchorX1, anchorY1
        idx0 = image_corners_df[image_corners_df['image_path'] == image_path0].index[0]
        image_corners_df.at[idx0, 'corners'] += [anchorX1, anchorY1]
        (anchorX, anchorY) = (0, 0)

    # Overlay warped image onto padded destination
    non_zero_mask = (warped_image > 0).astype(np.uint8)
    dst_padded[non_zero_mask == 1] = warped_image[non_zero_mask == 1]

    # Get last image's corners to crop the region of interest
    last_key = image_corners_df['image_path'].iloc[-1]
    warped_image_corners = image_corners_df.loc[image_corners_df['image_path'] == last_key, 'corners'].values[0]
    x_coords, y_coords = warped_image_corners[:, 0], warped_image_corners[:, 1]
    current_corners = np.array([
        [x_coords[0], y_coords[0]],
        [x_coords[1], y_coords[1]],
        [x_coords[2], y_coords[2]],
        [x_coords[3], y_coords[3]]
    ], dtype=np.float32)

    x_min, y_min = np.min(current_corners, axis=0).astype(int)
    x_max, y_max = np.max(current_corners, axis=0).astype(int)

    # warped_image = warped_image[y_min:y_max, x_min:x_max]
    # cv.imwrite(os.path.join(GENERAL_FOLDER_PATH, 'warped_image.jpg'), warped_image)
    image2 = load_image(image_paths[idx])
    image2 = rotate_image1(image2, -angle_deg)
    cv.imwrite(GENERAL_FOLDER_PATH, 'warped_image.jpg', cv.cvtColor(np.array(image2), cv.COLOR_RGB2BGR))
    cv.imwrite(PANORAMA_OUTPUT_PATH, dst_padded)

    if start:
        cv.imwrite(os.path.join(GENERAL_FOLDER_PATH, 'aligned_image.jpg'), cv.imread(PANORAMA_OUTPUT_PATH))

    # Load current panorama and aligned image
    current_panorama = cv.imread(PANORAMA_OUTPUT_PATH)
    new_image = cv.imread(os.path.join(GENERAL_FOLDER_PATH, 'aligned_image.jpg'))

    # Create translation matrix
    translation_matrix = np.float32([
        [1, 0, b_x_min - anchorX1],
        [0, 1, b_y_min - anchorY1],
        [0, 0, 1]
    ])

    if start:
        # For the first step, no translation needed
        translation_matrix = np.float32([
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1]
        ])

    dst_padded, warped_image, anchorX, anchorY, _ = warp_perspective_padded(current_panorama, new_image, translation_matrix)

    # Update DataFrame for all previous images (except the newly added one)
    idx_last = image_corners_df[image_corners_df['image_path'] == last_key].index[0]
    if not start:
        image_corners_df.at[idx_last, 'corners'] += [b_x_min - anchorX1 + anchorX, b_y_min - anchorY1 + anchorY]

    for img_path in image_corners_df['image_path']:
        if img_path != image_path1:
            idx_img = image_corners_df[image_corners_df['image_path'] == img_path].index[0]
            image_corners_df.at[idx_img, 'corners'] += [anchorX, anchorY]

    start = False

    # Combine images by filling zero pixels in dst_padded with warped_image pixels
    mask_dst = cv.cvtColor(dst_padded, cv.COLOR_BGR2GRAY)
    mask_dst = (mask_dst == 0).astype(np.uint8)
    mask_dst_3ch = cv.merge([mask_dst, mask_dst, mask_dst])
    combined_image = dst_padded.copy()
    combined_image[mask_dst_3ch == 1] = warped_image[mask_dst_3ch == 1]

    # Save the combined image as the newly aligned image
    cv.imwrite(os.path.join(GENERAL_FOLDER_PATH, 'aligned_image.jpg'), combined_image)

# Final panorama
cv.imwrite(os.path.join(GENERAL_FOLDER_PATH, 'final_panorama.jpg'), combined_image)


## Updated two way stitching

In [1]:
# %% [markdown]
# # Refactored Panorama Stitching Code

# %% [code]
import os
import glob
import math
import cv2 as cv
import cv2
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF

from lightglue import LightGlue, SuperPoint, viz2d, DISK, SIFT, ALIKED, DoGHardNet
from lightglue.utils import load_image, rbd, match_pair

# %% [code]
#######################################
# Global Configuration & Setup
#######################################
general_folder_path = '/home/oussama/Documents/EPFL/PDS_LUTS/'
output_path = os.path.join(general_folder_path, 'panorama.jpg')
image_folder = os.path.join(general_folder_path, 'images_updated')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

# Utility functions
def rotate_image(image, angle):
    """Rotate a PIL image tensor by a given angle."""
    return TF.rotate(image, angle)

def rotate_image_preserve(image: torch.Tensor, angle: float) -> torch.Tensor:
    """
    Rotate a PyTorch image tensor by a specified angle without cropping,
    adding necessary padding to preserve all content.
    """
    device = image.device
    image_np = image.permute(1, 2, 0).cpu().numpy()
    if image_np.max() <= 1:
        image_np = (image_np * 255).astype(np.uint8)

    h, w = image_np.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    cos_val = abs(rotation_matrix[0, 0])
    sin_val = abs(rotation_matrix[0, 1])

    new_w = int(h * sin_val + w * cos_val)
    new_h = int(h * cos_val + w * sin_val)

    rotation_matrix[0, 2] += (new_w / 2) - center[0]
    rotation_matrix[1, 2] += (new_h / 2) - center[1]

    rotated_image_np = cv2.warpAffine(
        image_np, rotation_matrix, (new_w, new_h),
        flags=cv2.INTER_CUBIC,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=(0, 0, 0)
    )

    rotated_image_tensor = torch.from_numpy(rotated_image_np).permute(2, 0, 1).float()
    if rotated_image_tensor.max() > 1:
        rotated_image_tensor /= 255.0

    return rotated_image_tensor.to(device)

def compute_similarity_score(image1, image2, extractor, matcher):
    """Compute similarity score between two images based on matched keypoints."""
    with torch.no_grad():
        feats0, feats1, matches01 = match_pair(extractor, matcher, image1, image2)
    points0 = feats0['keypoints'][matches01['matches'][..., 0]]
    score = points0.shape[0]
    del feats0, feats1, matches01, points0
    torch.cuda.empty_cache()
    return score

def split_image_paths(image_paths):
    """
    Splits the image_paths list into two halves:
    - First half gets an extra frame if the total number of images is odd.
    - Second half is reversed to start from the last image and go to the middle.

    Args:
        image_paths (list): List of image paths.

    Returns:
        tuple: (first_half, second_half) - two lists of image paths.
    """
    # Compute the midpoint
    mid = (len(image_paths) + 1) // 2

    # Split the list
    first_half = image_paths[:mid]  # First half
    second_half = image_paths[mid:][::-1]  # Second half, reversed

    return first_half, second_half

def warp_perspective_padded(src, dst, transf):
    """Warp 'src' image into 'dst' image using a homography 'transf' with padding."""
    src_h, src_w = src.shape[:2]
    dst_h, dst_w = dst.shape[:2]

    src_corners = np.float32([[0, 0],
                              [src_w, 0],
                              [src_w, src_h],
                              [0, src_h]])
    dst_corners = np.float32([[0, 0],
                              [dst_w, 0],
                              [dst_w, dst_h],
                              [0, dst_h]])

    src_corners_transformed = cv2.perspectiveTransform(src_corners[None, :, :], transf)[0]
    all_corners = np.vstack((src_corners_transformed, dst_corners))

    x_min, y_min = np.int32(all_corners.min(axis=0))
    x_max, y_max = np.int32(all_corners.max(axis=0))

    shift_x = -x_min
    shift_y = -y_min
    output_width = x_max - x_min
    output_height = y_max - y_min

    translation_matrix = np.array([
        [1, 0, shift_x],
        [0, 1, shift_y],
        [0, 0, 1]
    ], dtype=np.float32)

    new_transf = translation_matrix @ transf
    warped = cv2.warpPerspective(src, new_transf, (output_width, output_height))
    dst_pad = cv2.warpAffine(dst, translation_matrix[:2], (output_width, output_height))

    anchorX = int(shift_x)
    anchorY = int(shift_y)
    return dst_pad, warped, anchorX, anchorY

def split_image_paths(image_paths: list) -> (list, list):
    """
    Splits the image_paths list into two halves:
    - First half gets an extra frame if the total number of images is odd.
    - Second half is reversed (so that it starts from the last image and goes towards the middle).
    """
    mid = (len(image_paths) + 1) // 2
    first_half = image_paths[:mid]
    second_half = image_paths[mid:][::-1]
    return first_half, second_half

def choose_best_rotation(image0: torch.Tensor, image1: torch.Tensor, extractor, matcher, angles=range(0,360,45)):
    """Find the best rotation angle for image1 to match image0."""
    best_score, best_angle = -1, 0
    for angle in angles:
        rotated = rotate_image(image1, angle)
        score = compute_similarity_score(image0, rotated, extractor, matcher)
        if score > best_score:
            best_score, best_angle = score, angle
    return best_angle

def update_corners_df(image_corners_df: pd.DataFrame, image_path: str, corners: np.ndarray) -> pd.DataFrame:
    """Add a new row to the corners DataFrame with provided image path and corners."""
    frame_number = os.path.splitext(os.path.basename(image_path))[0]
    new_row = pd.DataFrame({
        'image_path': [image_path],
        'corners': [corners],
        'frame_number': [frame_number]
    })
    return pd.concat([image_corners_df, new_row], ignore_index=True)

def overlay_images(base_img: np.ndarray, overlay_img: np.ndarray) -> np.ndarray:
    """Overlay overlay_img onto base_img wherever base_img is zero."""
    mask_dst = cv2.cvtColor(base_img, cv2.COLOR_BGR2GRAY)
    mask_dst = (mask_dst == 0).astype(np.uint8)
    mask_dst_3ch = cv2.merge([mask_dst, mask_dst, mask_dst])
    combined = base_img.copy()
    combined[mask_dst_3ch == 1] = overlay_img[mask_dst_3ch == 1]
    return combined

In [2]:
#######################################
# Initialization
#######################################

# Initialize feature extractor and matcher
extractor = DoGHardNet(max_num_keypoints=None).eval().cuda()
matcher = LightGlue(features='doghardnet').eval().cuda()

# Get all .jpg files in the folder, sorted if necessary
image_paths = sorted(glob.glob(os.path.join(image_folder, '*.jpg')))
indice = 0
image_paths = image_paths[indice:] + image_paths[:indice]

# Load first image and set up DataFrame
image_path0 = image_paths[0]
cv.imwrite(os.path.join(general_folder_path, 'warped_image.jpg'), cv2.imread(image_path0))
image0 = load_image(image_path0)

image_corners_df = pd.DataFrame(columns=['image_path', 'corners', 'frame_number'])

first_image_corners = np.array([[0, 0],
                                [image0.shape[2]-1, 0],
                                [image0.shape[2]-1, image0.shape[1]-1],
                                [0, image0.shape[1]-1]], dtype=np.int32)

image_corners_df = update_corners_df(image_corners_df, image_path0, first_image_corners)
first_half, second_half = split_image_paths(image_paths)

In [3]:
#######################################
# Stitching First Half
#######################################
start = True

print("Processing first half of images...")
for idx in tqdm(range(indice + 1, 3)):
    if not start:
        image_path0 = os.path.join(general_folder_path, 'warped_image.jpg')
    image0 = load_image(image_path0)

    image_path1 = first_half[idx]
    image1 = load_image(image_path1)

    # Determine best rotation
    best_angle = choose_best_rotation(image0, image1, extractor, matcher)
    best_rotated_image = rotate_image(image1, best_angle)

    # Prepare images for final stitching
    imocv0 = cv2.imread(image_path0)
    imocv1 = cv2.cvtColor(np.array(TF.to_pil_image(best_rotated_image.cpu()).convert("RGB")), cv2.COLOR_RGB2BGR)

    # Match and compute homography
    feats0, feats1, matches01 = match_pair(extractor, matcher, image0, best_rotated_image)
    points0 = feats0['keypoints'][matches01['matches'][..., 0]].cpu().numpy()
    points1 = feats1['keypoints'][matches01['matches'][..., 1]].cpu().numpy()

    if points0.shape[0] < 4:
        print("Not enough points to compute homography.")
        continue

    M, _ = cv2.findHomography(points1, points0, cv2.RANSAC, 5.0)

    M_normalized = M / M[2, 2]
    M = np.array([
        [M_normalized[0, 0], M_normalized[0, 1], M_normalized[0, 2]],
        [M_normalized[1, 0], M_normalized[1, 1], M_normalized[1, 2]],
        [M_normalized[2, 0],                 0,                 1]
    ])

    angle_rad = math.atan2(M[1, 0], M[0, 0])
    angle_deg = math.degrees(angle_rad) - best_angle

    dst_padded, warped_image, anchorX1, anchorY1 = warp_perspective_padded(imocv1, imocv0, M)

    before_last_key = image_corners_df['image_path'].iloc[-1]
    prev_corners = image_corners_df.loc[image_corners_df['image_path'] == before_last_key, 'corners'].values[0]
    x_coords, y_coords = prev_corners[:, 0], prev_corners[:, 1]

    new_image_corners = np.array([[0, 0],
                                  [imocv1.shape[1], 0],
                                  [imocv1.shape[1], imocv1.shape[0]],
                                  [0, imocv1.shape[0]]], dtype=np.float32)

    transformed_corners = cv2.perspectiveTransform(np.array([new_image_corners], dtype=np.float32), M)[0]
    adjusted_corners = transformed_corners + [anchorX1, anchorY1]

    image_corners_df = update_corners_df(image_corners_df, image_path1, adjusted_corners)

    if start:
        idx0 = image_corners_df[image_corners_df['image_path'] == image_path0].index[0]
        image_corners_df.at[idx0, 'corners'] += [anchorX1, anchorY1]

    # Overlay images
    non_zero_mask = (warped_image > 0).astype(np.uint8)
    dst_padded[non_zero_mask == 1] = warped_image[non_zero_mask == 1]

    last_key = image_corners_df['image_path'].iloc[-1]
    last_corners = image_corners_df.loc[image_corners_df['image_path'] == last_key, 'corners'].values[0]
    lx, ly = last_corners[:, 0], last_corners[:, 1]
    x_min, y_min = np.min(np.array([lx, ly]), axis=1).astype(int)
    x_max, y_max = np.max(np.array([lx, ly]), axis=1).astype(int)

    # Crop the region of interest
    warped_image_cropped = warped_image[y_min:y_max, x_min:x_max]
    cv2.imwrite(os.path.join(general_folder_path, 'warped_image.jpg'), warped_image_cropped)
    cv2.imwrite(os.path.join(general_folder_path, 'panorama.jpg'), dst_padded)

    if start:
        cv2.imwrite(os.path.join(general_folder_path, 'aligned_image.jpg'),
                    cv2.imread(os.path.join(general_folder_path, 'panorama.jpg')))

    current_panorama = cv2.imread(os.path.join(general_folder_path, 'panorama.jpg'))
    new_image_aligned = cv2.imread(os.path.join(general_folder_path, 'aligned_image.jpg'))

    translation_matrix = np.float32([
        [1, 0, x_min - anchorX1],
        [0, 1, y_min - anchorY1],
        [0, 0, 1]
    ])

    if start:
        # For the first iteration, no translation is needed
        translation_matrix = np.float32([
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1]
        ])

    dst_padded_trans, warped_image_trans, anchorX, anchorY = warp_perspective_padded(current_panorama, new_image_aligned, translation_matrix)

    # Update corners
    idx_last = image_corners_df[image_corners_df['image_path'] == last_key].index[0]
    if not start:
        image_corners_df.at[idx_last, 'corners'] += [x_min - anchorX1 + anchorX, y_min - anchorY1 + anchorY]

    for img_path in image_corners_df['image_path']:
        if img_path != image_path1:
            idx_img = image_corners_df[image_corners_df['image_path'] == img_path].index[0]
            image_corners_df.at[idx_img, 'corners'] += [anchorX, anchorY]

    start = False

    combined_image = overlay_images(dst_padded_trans, warped_image_trans)
    cv2.imwrite(os.path.join(general_folder_path, 'aligned_image.jpg'), combined_image)


Processing first half of images...


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
#######################################
# Stitching Second Half
#######################################
indice = 0
start = True

print("Processing second half of images...")
for idx in tqdm(range(indice + 1, len(second_half))):
    # Determine the reference image (anchor) and the target image
    image_path0 = (general_folder_path + 'warped_image.jpg') if not start else image_paths[0]
    image_path1 = second_half[idx]

    image0 = load_image(image_path0)
    image1 = load_image(image_path1)

    # Find best rotation angle for image1 to match image0
    best_angle = choose_best_rotation(image0, image1, extractor, matcher)
    best_rotated_image = rotate_image(image1, best_angle)

    # Extract features and matches
    feats0, feats1, matches01 = match_pair(extractor, matcher, image0, best_rotated_image)
    points0 = feats0['keypoints'][matches01['matches'][..., 0]].cpu().numpy()
    points1 = feats1['keypoints'][matches01['matches'][..., 1]].cpu().numpy()

    if points0.shape[0] < 4:
        print("Not enough points to compute homography.")
        continue

    # Compute homography
    M, _ = cv.findHomography(points1, points0, cv.RANSAC, 5.0)

    # Normalize and adjust homography matrix
    M_normalized = M / M[2, 2]
    M_adjusted = np.array([
        [M_normalized[0, 0], M_normalized[0, 1], M_normalized[0, 2]],
        [M_normalized[1, 0], M_normalized[1, 1], M_normalized[1, 2]],
        [M_normalized[2, 0],             0,             1]
    ])
    M = M_adjusted

    # Compute angle in degrees
    angle_rad = math.atan2(M[1, 0], M[0, 0])
    angle_deg = math.degrees(angle_rad) + best_angle

    # Prepare OpenCV images
    imocv0 = cv.imread(image_path0)
    imocv1 = cv.cvtColor(np.array(TF.to_pil_image(best_rotated_image.cpu()).convert("RGB")), cv.COLOR_RGB2BGR)

    # Warp perspective
    dst_padded, warped_image, anchorX1, anchorY1 = warp_perspective_padded(imocv1, imocv0, M)

    # Retrieve corners from image_corners_df
    if start:
        before_last_key = image_corners_df['image_path'].iloc[0]
    else:
        before_last_key = image_corners_df['image_path'].iloc[-1]

    warped_image_corners = image_corners_df.loc[image_corners_df['image_path'] == before_last_key, 'corners'].values[0]
    x_coords, y_coords = warped_image_corners[:, 0], warped_image_corners[:, 1]

    corners = np.array([[x_coords[0], y_coords[0]],
                        [x_coords[1], y_coords[1]],
                        [x_coords[2], y_coords[2]],
                        [x_coords[3], y_coords[3]]], dtype=np.float32)

    b_x_min, b_y_min = np.min(corners, axis=0).astype(int)
    b_x_max, b_y_max = np.max(corners, axis=0).astype(int)

    # Define new image corners and transform them
    new_image_corners = np.array([[0, 0],
                                  [imocv1.shape[1], 0],
                                  [imocv1.shape[1], imocv1.shape[0]],
                                  [0, imocv1.shape[0]]], dtype=np.float32)

    transformed_corners = cv.perspectiveTransform(np.array([new_image_corners], dtype=np.float32), M)[0]
    adjusted_corners = transformed_corners + [anchorX1, anchorY1]

    # Update corners DF
    image_corners_df = update_corners_df(image_corners_df, image_path1, adjusted_corners)

    # Overlay warped image onto the padded destination
    non_zero_mask = (warped_image > 0).astype(np.uint8)
    dst_padded[non_zero_mask == 1] = warped_image[non_zero_mask == 1]

    # Get last and before last keys
    last_key = image_corners_df['image_path'].iloc[-1]
    warped_image_corners = image_corners_df.loc[image_corners_df['image_path'] == last_key, 'corners'].values[0]
    x_coords, y_coords = warped_image_corners[:, 0], warped_image_corners[:, 1]

    corners = np.array([[x_coords[0], y_coords[0]],
                        [x_coords[1], y_coords[1]],
                        [x_coords[2], y_coords[2]],
                        [x_coords[3], y_coords[3]]], dtype=np.float32)

    x_min, y_min = np.min(corners, axis=0).astype(int)
    x_max, y_max = np.max(corners, axis=0).astype(int)

    # Crop the region of interest
    cropped_warped_image = warped_image[y_min:y_max, x_min:x_max]
    cv.imwrite(general_folder_path + 'warped_image.jpg', cropped_warped_image)

    # Save current panorama
    cv.imwrite(general_folder_path + 'panorama.jpg', dst_padded)

    # Apply translation to align images
    current_panorama = cv.imread(general_folder_path + 'panorama.jpg')
    new_image = cv.imread(general_folder_path + 'aligned_image.jpg')  # Ensure this exists or handle it

    translation_matrix = np.float32([
        [1, 0, b_x_min - anchorX1],
        [0, 1, b_y_min - anchorY1],
        [0, 0, 1]
    ])

    dst_padded_tr, warped_image_tr, anchorX, anchorY = warp_perspective_padded(current_panorama, new_image, translation_matrix)

    # Update corners after translation
    idx_last = image_corners_df[image_corners_df['image_path'] == last_key].index[0]
    image_corners_df.at[idx_last, 'corners'] += [b_x_min - anchorX1 + anchorX, b_y_min - anchorY1 + anchorY]

    # Shift all other images except the current by the translation offset
    for img_path in image_corners_df['image_path']:
        if img_path != image_path1:
            idx_other = image_corners_df[image_corners_df['image_path'] == img_path].index[0]
            image_corners_df.at[idx_other, 'corners'] += [anchorX, anchorY]

    start = False

    # Combine images
    combined_image = overlay_images(dst_padded_tr, warped_image_tr)
    cv.imwrite(general_folder_path + 'aligned_image.jpg', combined_image)

# Save the final panorama
cv.imwrite(os.path.join(general_folder_path, 'final_panorama.jpg'), combined_image)

Processing second half of images...


  0%|          | 0/15 [00:00<?, ?it/s]

[ WARN:0@46.273] global loadsave.cpp:241 findDecoder imread_('/home/oussama/Documents/EPFL/PDS_LUTS/aligned_image.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
#######################################
# Display & Save Results
#######################################
plt.figure(figsize=(40, 30))
plt.imshow(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))
for _, row in image_corners_df.iterrows():
    corners = row['corners']
    x_coords, y_coords = corners[:, 0], corners[:, 1]
    plt.plot(x_coords, y_coords, 'o-', label=os.path.basename(row['image_path']))
    plt.fill(x_coords, y_coords, alpha=0.3)
plt.axis('on')
# plt.legend()  # Uncomment if you want a legend
plt.show()

image_corners_df.to_csv(os.path.join(general_folder_path, 'results/image_corners.csv'), index=False)
torch.cuda.empty_cache()